In [1]:
import pandas as pd
import json
import numpy as np
import re

# Construct JSON

In [41]:
df = pd.read_excel("./data_update.xlsx")
df

,就诊年龄,EMPI,szy自编序号,开方时间,中医药方,味数,剂数,频次,途径,开立医生,Unnamed: 10,Unnamed: 11,≥3诊的处方
0,54.0,16228.0,1.0,2014-02-11 17:18:00,盐山萸肉（制山萸肉、盐/制山茱萸）10g 熟地黄（熟地）15g 山药(淮山)15g 茯苓(云...,12.0,28.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
1,54.0,16228.0,1.0,2014-03-04 16:38:00,盐山萸肉（制山萸肉、盐/制山茱萸）10g 熟地黄（熟地）15g 山药(淮山)15g 茯苓(云...,12.0,28.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
2,54.0,16228.0,1.0,2014-04-01 19:20:00,盐山萸肉（制山萸肉、盐/制山茱萸）10g 知母15g 山药(淮山)15g 茯苓(云苓)15g...,13.0,22.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
3,54.0,16228.0,1.0,2014-04-29 17:51:00,盐山萸肉（制山萸肉、盐/制山茱萸）10g 山药(淮山)15g 茯苓(云苓)15g 牡丹皮10...,12.0,26.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
4,54.0,16228.0,1.0,2014-05-27 19:46:00,盐山萸肉（制山萸肉、盐/制山茱萸）10g 山药(淮山)15g 茯苓(云苓)15g 牡丹皮10...,13.0,28.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,28.0,2028375.0,33.0,2014-05-20 16:13:00,菟丝子（盐菟丝子）15g 金樱子（盐金樱子）15g 甘草（甘草粒）5g 土茯苓20g 桃仁（...,14.0,7.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
603,28.0,2028375.0,33.0,2014-06-10 16:06:00,菟丝子（盐菟丝子）15g 甘草（甘草粒）5g 桃仁（?桃仁）10g 黄芪(北芪)30g 首乌...,13.0,14.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
604,28.0,2028375.0,33.0,2014-08-12 18:49:00,黄芪(北芪)30g 生地黄(生地)15g 制何首乌15g 桃仁（?桃仁）10g 盐山萸肉（制...,12.0,7.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
605,28.0,2028375.0,33.0,2014-09-09 14:53:00,黄芪(北芪)30g 生地黄(生地)15g 首乌藤（夜交藤）20g 桃仁（?桃仁）10g 盐山...,12.0,7.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN


In [42]:
df = df.dropna(subset=['szy自编序号'])

In [43]:
df["szy自编序号"] = df["szy自编序号"].astype(int)
patients = df["szy自编序号"].unique()
patients

C:\Users\XinHe\AppData\Local\Temp\ipykernel_10448\2052352171.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["szy自编序号"] = df["szy自编序号"].astype(int)


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])

In [44]:
total = []
for n in patients:
    patient1 = df[df["szy自编序号"] == 1]
    data = patient1
    res_patient = []
    for idx, row in data.iterrows():
        res ={"visit": idx + 1}
        visit = idx + 1
        text = row["中医药方"]
        text_lst = text.split("g")
        for t in text_lst:
            t = t.strip().replace("（","(").replace("）",")")
            if t:
                # 使用正则表达式提取数字和名称
                match = re.search(r'(\d+)', t)
                if match:
                    num = int(match.group(1))
                    name = t[:match.start()].strip()
                    name = rename_map[name]
                    if name in res:
                        res[name] += num
                    else:
                        res[name] = num            
        res_patient.append(res)
    total.append(res_patient)
total

[[{'visit': 1,
   '山茱萸': 10,
   '熟地黄': 15,
   '山药': 15,
   '茯苓': 15,
   '牡丹皮': 10,
   '枳壳': 10,
   '佛手': 15,
   '茵陈': 15,
   '杜仲': 15,
   '桑寄生': 15,
   '延胡索': 15,
   '炙甘草': 5},
  {'visit': 2,
   '山茱萸': 10,
   '熟地黄': 15,
   '山药': 15,
   '茯苓': 15,
   '牡丹皮': 10,
   '杜仲': 15,
   '桑寄生': 15,
   '延胡索': 15,
   '炙甘草': 5,
   '白芍': 15,
   '佛手': 15,
   '黄精': 15},
  {'visit': 3,
   '山茱萸': 10,
   '知母': 15,
   '山药': 15,
   '茯苓': 15,
   '牡丹皮': 10,
   '黄精': 15,
   '杜仲': 15,
   '桑寄生': 15,
   '延胡索': 15,
   '甘草': 5,
   '佛手': 15,
   '布渣叶': 15,
   '络石藤': 15},
  {'visit': 4,
   '山茱萸': 10,
   '山药': 15,
   '茯苓': 15,
   '牡丹皮': 10,
   '杜仲': 15,
   '桑寄生': 15,
   '莲子': 15,
   '甘草': 5,
   '芡实': 15,
   '布渣叶': 15,
   '丹参': 15,
   '浙贝母': 15},
  {'visit': 5,
   '山茱萸': 10,
   '山药': 15,
   '茯苓': 15,
   '牡丹皮': 10,
   '杜仲': 15,
   '桑寄生': 15,
   '莲子': 15,
   '甘草': 5,
   '芡实': 15,
   '茜草': 10,
   '丹参': 15,
   '防风': 15,
   '白术': 15},
  {'visit': 6,
   '山茱萸': 10,
   '山药': 15,
   '茯苓': 15,
   '牡丹皮': 10,
   '杜仲': 15,
   '桑寄生': 1

In [45]:
with open("data_update_cleaned.json", "w") as f:
    json.dump(total, f, ensure_ascii=False, indent=2)

# 处理药名

In [30]:
info = pd.read_excel("四气五味+归经_703.xlsx")
info

,Chinese_name,Properties_Chinese,Meridians_Chinese
0,矮地茶,"辛,微苦,平","肺,肝"
1,艾叶,"辛,苦,温,有小毒","肝,脾,肾"
2,安息香,"辛,苦,平","心,脾"
3,八角茴香,"辛,温","肝,肾,脾,胃"
4,八角莲,"苦,辛,温,有毒","肺,肝"
...,...,...,...
700,鳖甲,"咸,微寒","肝,肾"
701,麝香,"辛,温","心,脾"
702,地黄,"甘,寒","心,肝,肾"
703,有瓜石斛,"甘,淡,凉","肺,胃,肾"


In [31]:
name_set = set(info["Chinese_name"])
len(name_set)

705

In [37]:
data = json.load(open("data_update.json"))
data[0][0]

{'visit': 1,
 '盐山萸肉(制山萸肉、盐/制山茱萸)': 10,
 '熟地黄(熟地)': 15,
 '山药(淮山)': 15,
 '茯苓(云苓)': 15,
 '牡丹皮': 10,
 '枳壳(炒枳壳)': 10,
 '佛手(制佛手)': 15,
 '茵陈(绵茵陈)': 15,
 '杜仲(盐杜仲)': 15,
 '桑寄生(杂寄生)': 15,
 '延胡索(醋延胡索/玄胡索/元胡/制延胡索)': 15,
 '炙甘草': 5}

In [38]:
import re

def split_synonyms(text):
    # 定义正则表达式模式
    pattern = r'([\u4e00-\u9fa5]+)[(（](.*?)[)）]'
    # 匹配主词和同义词部分
    match = re.match(pattern, text)
    if match:
        # 提取主词
        main_word = match.group(1)
        # 提取同义词部分
        synonyms_part = match.group(2)
        # 定义分隔符列表
        separators = r'[/、]'
        # 使用分隔符拆分同义词部分
        synonyms = re.split(separators, synonyms_part)
        # 将主词添加到同义词列表中
        all_synonyms = [main_word] + synonyms
        return all_synonyms
    return [text]

In [39]:
outilier = []
rename_map = dict()
for patient in data:
    for visit in patient:
        # 找出keys
        herbs = set(visit.keys()) - {"visit"}
        for herb in herbs:
            names = split_synonyms(herb)
            flag = False
            for name in names:
                if name in name_set:
                    flag = True
                    rename_map[herb] = name
                    break
            if not flag:
                outilier.append(herb)
set(outilier)

{'干石斛(流苏石斛)',
 '广东王不留行',
 '炒白术',
 '炒薏苡仁(炒苡米)',
 '盐山萸肉(制山萸肉、盐/制山茱萸)',
 '蜜款冬花(炙冬花)',
 '金樱子肉'}

In [40]:
rename_map["干石斛(流苏石斛)"] = "石斛"
rename_map["炒薏苡仁(炒苡米)"] = "薏苡仁"
rename_map["广东王不留行"] = "王不留行"
rename_map["炒白术"] = "白术"
# add 有瓜石斛
rename_map['盐山萸肉(制山萸肉、盐/制山茱萸)'] = '山茱萸'
# add 茯神
rename_map['蜜款冬花(炙冬花)'] = "款冬花"
rename_map['金樱子肉'] = "金樱子"

{'visit': 1,
 '山茱萸': 10,
 '熟地黄': 15,
 '山药': 15,
 '茯苓': 15,
 '牡丹皮': 10,
 '枳壳': 10,
 '佛手': 15,
 '茵陈': 15,
 '杜仲': 15,
 '桑寄生': 15,
 '延胡索': 15,
 '炙甘草': 5}

# 读取新数据中没有分类的

In [47]:
data = json.load(open("data_update_cleaned.json"))
data[0][0]

{'visit': 1,
 '山茱萸': 10,
 '熟地黄': 15,
 '山药': 15,
 '茯苓': 15,
 '牡丹皮': 10,
 '枳壳': 10,
 '佛手': 15,
 '茵陈': 15,
 '杜仲': 15,
 '桑寄生': 15,
 '延胡索': 15,
 '炙甘草': 5}

In [48]:
# 遍历data中的patient和visit，找出所有的药名
herbs = set()
for patient in data:
    for visit in patient:
        herbs.update(visit.keys())
herbs = herbs - {"visit"}
herbs

{'丹参',
 '乌药',
 '佛手',
 '地肤子',
 '墨旱莲',
 '女贞子',
 '射干',
 '山茱萸',
 '山药',
 '布渣叶',
 '广藿香',
 '延胡索',
 '有瓜石斛',
 '杜仲',
 '枳壳',
 '枸杞子',
 '柴胡',
 '桑寄生',
 '桔梗',
 '款冬花',
 '法半夏',
 '浙贝母',
 '炙甘草',
 '熟地黄',
 '牛膝',
 '牡丹皮',
 '王不留行',
 '甘草',
 '白术',
 '白芍',
 '白茅根',
 '白鲜皮',
 '百部',
 '益智',
 '知母',
 '石斛',
 '石韦',
 '粉葛',
 '紫苏叶',
 '络石藤',
 '芡实',
 '茜草',
 '茯神',
 '茯苓',
 '茵陈',
 '荠菜',
 '莲子',
 '蒺藜',
 '薏苡仁',
 '藕节',
 '金樱子',
 '防风',
 '首乌藤',
 '黄精'}

In [49]:
table_json = json.load(open("Table.json"))
table_json

[{'药名': '茜草',
  '性味归经': '苦、寒。归肝经。',
  '功效': '凉血、祛瘀、止血、通经',
  '主治': '用于吐血、衄血、崩漏、外伤出血、瘀阻经闭、关节痹痛、跌扑肿痛',
  '教材分类': '止血药',
  '名医对于药物的用法': '清热止血、治血尿',
  '专家分类': '凉血止血类'},
 {'药名': '仙鹤草',
  '性味归经': '苦、涩、平。归心、肝经。',
  '功效': '收敛止血、截疟、止痢、解毒、补虚',
  '主治': '用于咯血、吐血、崩漏下血、疟疾、血痢、痈肿疮毒、阴痒带下、脱力劳伤',
  '教材分类': '止血药',
  '名医对于药物的用法': '止血、治血尿',
  '专家分类': '凉血止血类'},
 {'药名': '小蓟',
  '性味归经': '甘、苦、凉。归心、肝经。',
  '功效': '凉血止血、散瘀解毒消痈',
  '主治': '用于衄血、吐血、尿血、血淋、便血、崩漏、外伤出血、痈肿疮毒',
  '教材分类': '止血药',
  '名医对于药物的用法': '清热止血、治血尿',
  '专家分类': '凉血止血类'},
 {'药名': '合欢皮',
  '性味归经': '甘、平。归心、肝、肺经。',
  '功效': '解郁安神、活血消肿',
  '主治': '用于心神不安、忧郁失眠、肺痈、疮肿、跌扑伤痛',
  '教材分类': '养心安神药',
  '名医对于药物的用法': '疏肝、安神',
  '专家分类': '安神类'},
 {'药名': '酸枣仁',
  '性味归经': '甘、酸、平。归肝、胆、心经。',
  '功效': '养心补肝、宁心安神、敛汗、生津',
  '主治': '用于虚烦不眠、惊悸多梦、体虚多汗、津伤口渴',
  '教材分类': '养心安神药',
  '名医对于药物的用法': '安神敛阴',
  '专家分类': '安神类'},
 {'药名': '大黄',
  '性味归经': '苦、寒。归脾、胃、大肠、肝、心包经。',
  '功效': '泻下攻积、清热泻火、凉血解毒、逐瘀通经、利湿退黄',
  '主治': '用于实热积滞便秘、血热吐衄、目赤咽肿、痈肿疔疮、肠痈腹痛、瘀血经闭、产后瘀阻、跌打损伤、湿热痢疾、黄疸尿赤、淋证、水肿；外治烧烫伤',
  '教材分类': '泻

In [50]:
table_herbs = set([item["药名"] for item in table_json])
table_herbs

{'三七',
 '丹参',
 '五指毛桃',
 '仙鹤草',
 '党参',
 '制何首乌',
 '合欢皮',
 '土茯苓',
 '地肤子',
 '墨旱莲',
 '大黄',
 '太子参',
 '女贞子',
 '小蓟',
 '山茱萸',
 '山药',
 '延胡索',
 '当归',
 '板蓝根',
 '桃仁',
 '桑寄生',
 '泽兰',
 '淫羊藿',
 '熟地黄',
 '牛膝',
 '甘草',
 '生地黄',
 '白术',
 '白芍',
 '白花蛇舌草',
 '白茅根',
 '白鲜皮',
 '知母',
 '石韦',
 '砂仁',
 '积雪草',
 '茜草',
 '茯苓',
 '菊花',
 '菟丝子',
 '蒲公英',
 '薏苡仁',
 '赤芍',
 '车前子',
 '酸枣仁',
 '重楼(七叶一枝花）',
 '金银花',
 '陈皮',
 '鱼腥草',
 '麦芽',
 '黄精',
 '黄芩',
 '黄芪'}

In [51]:
# 找出data中的药名在table中没有的药名
outliers = herbs - table_herbs
outliers

{'乌药',
 '佛手',
 '射干',
 '布渣叶',
 '广藿香',
 '有瓜石斛',
 '杜仲',
 '枳壳',
 '枸杞子',
 '柴胡',
 '桔梗',
 '款冬花',
 '法半夏',
 '浙贝母',
 '炙甘草',
 '牡丹皮',
 '王不留行',
 '百部',
 '益智',
 '石斛',
 '粉葛',
 '紫苏叶',
 '络石藤',
 '芡实',
 '茯神',
 '茵陈',
 '荠菜',
 '莲子',
 '蒺藜',
 '藕节',
 '金樱子',
 '防风',
 '首乌藤'}